In [2]:
##
## ---------------------------------------------------------------------------
##

# Import SparkSession
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, FloatType, StructType, StructField, IntegerType,  StringType

## the distance function
from scipy.spatial import distance
## creates the feature vector
from pyspark.ml.feature import VectorAssembler
## import numpy
import numpy as np

from pyspark.sql.functions import max, abs, pow, sum, col

from pyspark.sql.functions import greatest

import time

import pyspark.sql.functions as F

from statistics import mode

import matplotlib.pyplot as plt

from functools import reduce

from pyspark.ml.linalg import Vectors

import numpy as np

import heapq


## start session
spark = SparkSession.builder.appName("SparkLAESAKnn").getOrCreate()




##DEFINES A UDF FROM L2DIST THE DATAFRAME FROM A QUERY SET
distance_udf = F.udf(lambda x,y: float(distance.euclidean(x, y)), DoubleType())

def simpleF(oq):
    return F.udf(lambda x: float(distance.euclidean(x, oq)), DoubleType())

## defines query object and 
#oq = [np.random.rand(1)[0],np.random.rand(1)[0]]
oq = [(1.0,[0.312,0.792])]
oqColumns = ["idOq","fvOq"]
oqDF = spark.createDataFrame(data=oq, schema = oqColumns)
#oqDF.show()


## defines neighbrs amount
k = 10

# define pivots amount
pivots = 2

nPivots = 2

# df
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("coord_x", FloatType(), True),
    StructField("coord_y", FloatType(), True)])

df = spark.read.csv("/home/weber/Documents/coordDF100.csv",schema=schema)        


size = df.count()
##SMALL SANITY CHECK - @PRODUCTION TESTAR LOADING
df = df.na.drop()

## define the struct for the dimensional feature vector
cNames = df.columns
cNames.remove("id")
assembler = VectorAssembler(
    inputCols=cNames,
    outputCol="fv")

## appends the fv into the dataframe as column
df = assembler.transform(df)


# creates random pivots
pivots_list = [(i+1,[np.random.rand(1)[0],np.random.rand(1)[0]]) for i in range(pivots)]

## calculates the distance from oi to pivots
for i in range(len(pivots_list)):
    df = df.withColumn(f'distances_oi_pivot{i+1}', simpleF(pivots_list[i][1])(F.col('fv')))

    
##
##
##
lista_columns = []

rows_oqDF = oqDF.collect()

for oq in rows_oqDF: 
    ##
    ## DISTANCE FROM OQ TO PIVOT
    ##
    lista_columns = []
    for i in range(len(pivots_list)):
        df = df.withColumn(f"lower_oq_{int(oq.idOq)}_pivot_{i+1}", \
            abs (distance.euclidean(oq.fvOq, pivots_list[i][1]) - \
            F.col(f'distances_oi_pivot{i+1}')))

        lista_columns.append(f"lower_oq_{int(oq.idOq)}_pivot_{i+1}")

    df = df.withColumn(f"lower_bound_oq_{int(oq.idOq)}", greatest(*[col_name for col_name in lista_columns]))


df.show()


+---+-------+-------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+
| id|coord_x|coord_y|                  fv|distances_oi_pivot1|distances_oi_pivot2|  lower_oq_1_pivot_1|  lower_oq_1_pivot_2|    lower_bound_oq_1|
+---+-------+-------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+
| 85|0.76609|0.64468|[0.76608997583389...|0.41778435337450587| 0.8028971733507899| 0.20707898361741833| 0.46906233987117324| 0.46906233987117324|
| 86|0.26943|0.14308|[0.26943001151084...| 0.3719507735888958| 0.8703225256674223|  0.2529125634030284|  0.5364876921878057|  0.5364876921878057|
| 87|0.14337|0.68237|[0.14337000250816...| 0.6465445658220822|0.31772170702902175|0.021681228830157995|0.016113126450594928|0.021681228830157995|
| 88|0.50937|0.54759|[0.50937002897262...| 0.3181158510618258| 0.6413638134022074|  0.3067474859300984|  0.3075289799225907|

In [ ]:
import matplotlib.pyplot as plt
import pyspark.sql.functions as F

# Assuming your DataFrame is named `df`
#x_col = 'coord_x'
#y_col = 'coord_y'

# Extract the x and y coordinates from the DataFrame
x = df.select("coord_x").rdd.flatMap(lambda x: x).collect()
y = df.select("coord_y").rdd.flatMap(lambda x: x).collect()

# Plot the points on a Cartesian plane
plt.scatter(x, y)
plt.xlabel("x")
plt.ylabel("y")
plt.show()


In [ ]:
def calculate_sum_of_squares(df,size):
    return df.groupBy("features", "avg(distance)").agg(
        (sum(pow(col("distance") - col("avg(distance)"), 2))/size).alias("variance")
    )

def maxVariance(df, nPivots):
    # create sample dataset
    dataset = df.select("id","coord_x","coord_y")

    # create a random sample of the input dataset
    sample1 = dataset.sample(False, 0.1, seed=42)
    sample2 = dataset.sample(False, 0.1, seed=24)

    # convert dataset to vectors
    cNames = dataset.columns
    cNames.remove("id")
    assembler = VectorAssembler(inputCols=cNames,outputCol="features")


    #assembler = VectorAssembler(inputCols=dataset.columns, outputCol="features")
    dataset_vectors = assembler.transform(dataset)
    
    dataset_vectors.show()
    
    sample_vectors1 = assembler.transform(sample1)
    sample_vectors2 = assembler.transform(sample2).withColumnRenamed("features","features2")

    sample_vectors2.show()
    # calculate mean distance between objects in part1 and part2
    cross_df = sample_vectors1.crossJoin(sample_vectors2.select("features2"))

    cross_df.show()
    
    distance = cross_df.withColumn("distance", distance_udf("features", "features2"))

    mean_distance = distance.groupBy("features").agg({"distance": "mean"})

    new_df = mean_distance.join(distance, on="features", how="inner")

    variance_df = calculate_sum_of_squares(new_df,(sample_vectors2.count()+1))
    rows = variance_df.select("features", "variance").collect()
    list_var = [(row.features, row.variance) for row in rows]
    sorted_lst = sorted(list_var, key=lambda x: x[1], reverse=True)[0:nPivots]
    
    return(sorted_lst)

maxVariance(df, nPivots)

In [3]:
class MaxVariance:
    
    def __init__(self, df, nPivots):
        self.df = df
        self.nPivots = nPivots
        
    def distance_udf(self):
        return F.udf(lambda x, y: float(distance.euclidean(x, y)), DoubleType())
    
    def calculate_sum_of_squares(self, df, size):
        return df.groupBy("features", "avg(distance)").agg(
            (sum(pow(col("distance") - col("avg(distance)"), 2))/size).alias("variance")
        )
    
    def find_max_variance(self):
        # create sample dataset
        dataset = self.df.select("id","coord_x", "coord_y")

        # create a random sample of the input dataset
        sample1 = dataset.sample(False, 0.1, seed=42)
        sample2 = dataset.sample(False, 0.1, seed=24)

        # convert dataset to vectors
        cNames = dataset.columns
        cNames.remove("id")
        assembler = VectorAssembler(inputCols=cNames,outputCol="features")
        
        
        #assembler = VectorAssembler(inputCols=dataset.columns, outputCol="features")
        dataset_vectors = assembler.transform(dataset).select("features")
        sample_vectors1 = assembler.transform(sample1).select("features")
        sample_vectors2 = assembler.transform(sample2).select("features").withColumnRenamed("features","features2")

        # calculate mean distance between objects in part1 and part2
        cross_df = sample_vectors1.crossJoin(sample_vectors2)
        distance = cross_df.withColumn("distance", self.distance_udf()("features", "features2"))
        mean_distance = distance.groupBy("features").agg({"distance": "mean"})
        
        # calculate variance between objects in part1 and part2
        new_df = mean_distance.join(distance, on="features", how="inner")
        variance_df = self.calculate_sum_of_squares(new_df, (sample_vectors2.count() + 1))
        
        # save the variances and sort it
        rows = variance_df.select("features", "variance").collect()
        list_var = [(row.features, row.variance) for row in rows]
        sorted_lst = sorted(list_var, key=lambda x: x[1], reverse=True)[0:self.nPivots]
        
        return sorted_lst
    
    def find_max_variance_fullset(self):
        # create sample dataset
        dataset = self.df.select("id","coord_x", "coord_y")

        # create a random sample of the input dataset
        sample1 = dataset
        sample2 = dataset

        # convert dataset to vectors
        cNames = dataset.columns
        cNames.remove("id")
        assembler = VectorAssembler(inputCols=cNames,outputCol="features")
        
        
        #assembler = VectorAssembler(inputCols=dataset.columns, outputCol="features")
        dataset_vectors = assembler.transform(dataset).select("features")
        sample_vectors1 = assembler.transform(sample1).select("features")
        sample_vectors2 = assembler.transform(sample2).select("features").withColumnRenamed("features","features2")

        # calculate mean distance between objects in part1 and part2
        cross_df = sample_vectors1.crossJoin(sample_vectors2)
        distance = cross_df.withColumn("distance", self.distance_udf()("features", "features2"))
        mean_distance = distance.groupBy("features").agg({"distance": "mean"})
        
        # calculate variance between objects in part1 and part2
        new_df = mean_distance.join(distance, on="features", how="inner")
        variance_df = self.calculate_sum_of_squares(new_df, (sample_vectors2.count() + 1))
        
        # save the variances and sort it
        rows = variance_df.select("features", "variance").collect()
        list_var = [(row.features, row.variance) for row in rows]
        sorted_lst = sorted(list_var, key=lambda x: x[1], reverse=True)[0:self.nPivots]
        
        return sorted_lst


In [4]:
max_variance_obj = MaxVariance(df, nPivots)
max_variance = max_variance_obj.find_max_variance()
max_variance2 = max_variance_obj.find_max_variance_fullset()
print(max_variance)
print(max_variance2)

[(DenseVector([0.8887, 0.3965]), 0.09519428005514337), (DenseVector([0.1539, 0.2099]), 0.08109210623909925)]
[(DenseVector([0.1561, 0.0042]), 0.08716387386690382), (DenseVector([0.1582, 0.0275]), 0.08606545175876348)]
